# Lecture Notes

Instructions:

> **Week k (week that group topics are presented in lecture)** <br>
By Friday at 4pm, the group is required to submit their typed-up notes from that week’s
lecture. This will form the basic outline of their deliverable and they will receive
feedback from the TA on their work. <br><br>
The group should also have an outline of the one outside paper that they intend to
describe in greater detail.

## L13. Constraint Programming

### Big picture

Constraint programming (CP) is a powerful tull to solve combinatorial optimization problems (e.g. planning, scheduling system design etc). Constraints arise in most areas of human endeavour. They basically formalize the relation among several unknowns (or variables), each taking a value in a given domain. The constraint thus restricts the possible values that variables can take, it represents partial information about the variables of interest. For instance, _"The three angles of a triangle sum to 180 degrees"_ relates each angle of the triangle without precisely specifying their values.

Constraint programming is the study of computational systems based on constraints. The idea of constraint programming is to solve problems by stating constraints (requirements) about the problem area and, consequently, finding solution satisfying all the constraints.

A Constraint Satisfaction Problems is usually referred as **CSP**, but it is worth noticing that any CSP can be trasformed in an optimization problem by simpy adding an objective function.

### Viewing Sodoku and N-Queens as CSPs

**N-Queens** The N-queens problem is a well know puzzle among computer scientists. Given any integer $N$, the problem is to place $N$ queens on a $N\times N$ chessboard satisfying the constraint that no two queens threaten each other.

<img src="images/nqueens.png" align="center"/>

Where a queen threatens any other queens on the same row, column and diagonal.

**Sudoku** Another well known puzzle that can be easly solved as CPSs is the Sudoku puzzle. Given a $9\times 9$ grid partially filled with numbers from $1$ to $9$, the goal is to fill the grid entirely such that:

- each $3\times 3$ sub-grid contains all number fomr $1$ to $9$ once
- each row contains all number fomr $1$ to $9$ once
- each column contains all number fomr $1$ to $9$ once

<table><tr>
<td> <img src="images/start_sudoku.png"/> </td>
<td> <img src="images/end_sudoku.png" /> </td>
</tr></table>

### How CSPs are (formally) defined

With the two examples from above it is easy to understand how a constraint programming problem is defined. A **CSP** is defined by the triple $\langle X,\,D,\,C \rangle$, where
- $X=\{x_1,x_2,\ldots,x_n\}$ is the set of _variables_
- $D=\{D_1,D_2,\ldots,D_n\}$ is the set of _variable domains_, where $x_i\in D_i$ for each $i\in\{ 1,\ldots,n\}$
- $C=\{C_1,C_2,\ldots,C_k\}$ is the set of _constraints_
    - A Constraint $C_i = \langle S_i,\, R_i \rangle$ specifies the allowed variable assignment, where
    - The _scope_ $S_i$ is the set of variables it works on
    - The _relation_ $R_i$ is a set of vilid assignments for $S_i$

### Applications of CSPs

TODO

### Tying to other course content
TODO

## L14. Solving Constraint Programs - Propagation and Basic Search
- Constraint propagation: Achieving arc consistency with AC-1 and AC-3
- Examples on integer domains with algebra
- Complexity of AC-1 and AC-3
- Arc consistency: sound but not complete

## L15. Solving Constraint Programs - Backtrack Search and Forward Checking
- Narrative: show the largest reasonable n-Queens problem that can be solved for various solution methods
- Combining inference with search
    - Why standard search sucks
    - Backtrack search
- Constraint Optimization Problem: Branch & Bound
- Forward Checking: Combining backtracking with **limited** constraint propagation
- Dynamic variable ordering and BT-FC
- "Bucket Elimination", a form of adaptive consistency (i.e., variable elimination)
    - Viewing arc consistency as variable elimination

###### Optional Content, Not Discussed in Lecture
- Adaptive consistency is linear for trees
- Iterative repair
- Conflict-directed back jumping